# Data Preprocessing
jumlah dan kualitas data adalah kunci yang mentukan seberapa baik sebuah algoritma machine learing dapat belajar. Oleh karena itu sangat penting untuk mengextract dan melakukan prepreocessing data sebelum data diproses oleh algoritma pembelajaran pada notebook ini akan dibahas mengenai beberapa topik mengenai preprocessing data diantaranya
* Menghilangkan dan memasukan missing value dari dataset
* Mengubah data keategorikal agar bisa diproses oleh algoritma machine learning
* Menentukan feature yang paling relevan untuk pembangunana model


## Menghilangkan dan memasukan missing value dari dataset
cara paling mudah untuk mengatasi data yang hilang adalah menghilangkan data tersebut, misal kita memiliki data yang memiliki beberapa feature missing, hilangkan saja feature tersebut dari dataset. untuk dataset yang sedikit kita bisa langsung saja meilihat feautre yang memiliki missing value namun untuk dataset yang berukuran besar kita harus mengecek satu persatu feature yang mengalami missing value dan menjumlahkannya. Panda dataframe menyediakan method ```isnull``` yang akan mereturn dataframe dengan nilai ```boolean``` dengan menggunakan method ```sum``` kita bisa mengetahui statistik missing values per feature. berikut ini adalah contohnya 

In [1]:
import pandas as pd
from io import StringIO

csv_data = '''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [3]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

In [4]:
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


Namun seringnnya, menghilangkan data sample atau menghilangkan feature dari dataset bukanlah suatu cara yang layak, karena bisa menyebabkan kehilangan banyak informasi dari data atau feature yang dihilangkan. Untuk mengatasi masalah ini kita bisa mengunakan teknik interpolation untuk mengestimasi nilai dari data yang mengalami missing value. Teknik interpolation yang paling banyak digunakan adalah ***mean imputation***. Cara ini bisa dibilang cukup mudah karena kita mengganti feature yang mengalami missing value dengan nilai rata-rata feature tersebut. Berikut ini adalah implementasi mean imputation dengan menggunakan scikit-learn

In [5]:
from sklearn.preprocessing import Imputer

impr = Imputer(missing_values='NaN', strategy='mean', axis=0)
impr = impr.fit(df)
imputed_data = impr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

## Mengatasi data categorical
pada umumnya data dibagi menjadi dua jenis yaitu ***nominal*** dan ***ordinal*** dimana nominal adalah data yang sifatnya hanya membedakan jenisnya saja dan biasanya tidak bisa diurutkan misalnya warna baju, sednagkan ordinal adalah data yang membedakan jenisnya berdasarkan urutannya shingga biasanya data tipe ordinal terurut misalnya ukuran baju. Sebelum memeahami prepreocessing terhadap data categorical baikya kita membuat dataset untuk data categorical

In [6]:
df = pd.DataFrame([
    ['hijau', 'M', 10.1, 'label1'],
    ['merah', 'L', 13.5, 'label2'],
    ['biru', 'XL', 15.3, 'label1']
])
df.columns = ['warna', 'ukuran', 'harga', 'labelkelas']
df

,warna,ukuran,harga,labelkelas
0,hijau,M,10.1,label1
1,merah,L,13.5,label2
2,biru,XL,15.3,label1


### Mapping ordinal feature
untuk memastikan bahwa algoritma learning mengintepretasikan ordinal feature secara tepat. perlu dilakukan konversi data string ke integer. pada scikit-learn tidakmenyediakn method ataupun fungsi yang secara otomatis mengurutkan feature dari ukuran baju sehingga perlu dilakukan mapping secara manual contohnya $XL = L + 1 = M + 2$, berikut ini adalah potongan kode untuk melakukan mapping ukuran baju

In [7]:
ukuran_mapping = {
    'XL': 3,
    'L': 2,
    'M' : 1
}

df['ukuran'] = df['ukuran'].map(ukuran_mapping)
df

,warna,ukuran,harga,labelkelas
0,hijau,1,10.1,label1
1,merah,2,13.5,label2
2,biru,3,15.3,label1


### Encoding label kelas
Banyak library machine learning yang membutuhkan lebel kelas yang di encode dengan nilai integer. Untuk melakukan encoding label kelas kita bisa melakukan pendekatan dengan cara yang sama saat melakukan mapping ordinal feature. Hal yang perlu kita ingat adalah bahwa label data bukan bersifat ordinal. Sehingga tidak akan menjadi masalah nomor yang akan ditetapkan untuk label, oleh karena itu kita mulai melakukan enumerasi label dari angka 0 

In [8]:
import numpy as np

label_mapping = {
    label: idx for idx, label in enumerate(np.unique(df['labelkelas']))
}
print(label_mapping)

df['labelkelas'] = df['labelkelas'].map(label_mapping)
df

{'label1': 0, 'label2': 1}


,warna,ukuran,harga,labelkelas
0,hijau,1,10.1,0
1,merah,2,13.5,1
2,biru,3,15.3,0


cara alternatif untuk melakukan encoding adalah dengan menggunakan ```LabelEncoder``` yang disediakan oleh library scikit-learn. Beriut adalah implementasi dari ```LabelEncoder```

In [9]:
from sklearn.preprocessing import LabelEncoder

kelas_le = LabelEncoder()
y = kelas_le.fit_transform(df['labelkelas'].values)
y

array([0, 1, 0])

### melakukan one-hot encoding pada nominal feature
Pembahasan-pembahasan sebelummnya kita sudah memahami bagaimana melakukan konversi data ordinal ke integer dengan menggunakan pendekatan dictionary-mapping. Dengan menggunakan ```LabelEncoder``` yang disediakan oleh library scikit-learn kita bisa melakukan encoding data string menjadi integer

In [10]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)
df_wine.columns = ['Class label', 'Alcohol',
                  'Malic Acid', 'Ash',
                  'Alcalinity of ash', 'Magnesium',
                  'Total phenols', 'Flavoids',
                  'Nonflavanoid phenols',
                  'Proanthocyanins',
                  'Color Intensity', 'Hue',
                  'OD280/OD315 diluted wines',
                  'Proline']
print('Class labels', np.unique(df_wine['Class label']))
df_wine.head()

Class labels [1 2 3]


,Class label,Alcohol,Malic Acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavoids,Nonflavanoid phenols,Proanthocyanins,Color Intensity,Hue,OD280/OD315 diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [11]:
from sklearn.cross_validation import train_test_split

X,y = df_wine.iloc[:,1:].values, df_wine.iloc[:,0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

/home/blacksmith/.virtualenvs/Machine_Learning/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Feature scaling
Ada dua pendekatan feature scaling yang paling banyak digunakan yaitu ***normalisasi*** dan ***standarisasi***. Pada normalisasi  adalah untuk melakukan pengsekalaan ulang agar feature yang diadapt berada diantara 0 hingga 1, dimana normalisasi merupakan spesial case dari min-max scaling. Untuk melakukan normalisasi data kita perlu mengaplikasikan min-max scaling untuk tiap ffeature, dimana nilai normalisasi $x_{norm}^{i}$ dari data sample $x^{j}$ dapat dihitung dengan menggunakan persamaan

$$x_{norm}^{(i)} = \frac{x^{(i)} - x_{min}}{x_{max}-x_{min}}$$

dimana $x^{(i)}$ adalah data sample, $x_{min}$ adalah nilai feature terkecil dan $x_{max}$ adalah nilai feature terbesar. Berikut ini adalah implementasi normalisasi pada scikit-learn 

In [12]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
x_train_norm = mms.fit_transform(X_train)
x_test_norm = mms.transform(X_test)

Standarisasi adalah salah satu metode yang praktikal digunakan pada algoritma machine learning dikarenakan banyak linear model sperti regresi logistik, SVM, dan sebagainya menginisialisasikan nilai random weight mendekati nilai 0. Dengan menggunakan standarisasi kita memusatkan nilai rata-rata feature menjadi 0 dengan standar deviasi 1, sehingga nilai feature akan terdistribusi secara normal, sehingga memudahkan untuk melakukan learing weight. Untuk melakukan standarisasi bisa dilakukan dengan menggunakan persamaan

$$x_{std}^{(i)} = \frac{x^{(i)}-\mu_x}{\sigma_x}$$

dimana $\mu$ adalah nilai rata-rata feature kolom, dan $\sigma$ adalah nilai untuk standar deviasi. 

Berikut ini adalah tabel yang membedakan antara normalisasi dengan standari sasi dimana nilai input yang digunakan adalah bilangan integer dari 0 hingga 4

| Input | standarisasi | Normalisasi |
| ------|:------------:| -----------:|
| 0.0   | -1.336306    | 0.0         |
| 1.0   | -0.801784    | 0.2         |
| 2.0   | -0.267261    | 0.4         |
| 3.0   | 0.267261     | 0.6         |
| 4.0   | 0.801784     | 0.8         |
| 5.0   | 1.336306     | 1.0         |

Berikut ini adalah implementasi standarisasi dengan menggunakan library scikit-learn

In [13]:
from sklearn.preprocessing import StandardScaler

stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.fit_transform(X_test)

## Memilih feature yang paling relevan
ketika model machine learning yang kita gunakan performanya lebih bagus pada data training dibanding pada data testing hal ini menunjukan bahwa model yang kita gunakan mengalamai ***overfitting***, alasan terjadinya overfitting adalah kerana model yang digunakan terlalu kompleks mengklasifikasikan data training, berikut ini adalah beberapa cara untuk mengatasi masalah overfitting
* kumpulkan data training lagi
* pilih model dengan parameter lebih sedikit
* gunakan regularisasi
* kurangi dimensi data
pada bagian ini dibahas mengani dua cara yaitu mengatasi overfitting dengan regularisasi dan mengurangi dimensi data

### L1 Regularisasi
Sebenarnya ada dua pendekatan regulasi yang bisa digunakan untuk mengatasi masalah overfitting yaitu ***L2 regularisasi*** dan ***L1 regularisasi***.
keduanya memiliki fungsiyang sama yaitu mereduksi kompleksitas suatu model.

In [14]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l1', C=0.1)
lr.fit(X_train_std, y_train)
print('Training accuracy:',lr.score(X_train_std, y_train))
print('Test accuracy:',lr.score(X_test_std, y_test))

#print intercept
print('intercept:',lr.intercept_)

#print OvR(One-vs-Rest)
print("One-vs-Rest \n",lr.coef_)

Training accuracy: 0.9838709677419355
Test accuracy: 0.9814814814814815
intercept: [-0.38380163 -0.15806079 -0.70044752]
One-vs-Rest 
 [[ 0.28005163  0.          0.         -0.02811601  0.          0.
   0.70994441  0.          0.          0.          0.          0.
   1.23657375]
 [-0.64411304 -0.06874675 -0.05721604  0.          0.          0.
   0.          0.          0.         -0.92647981  0.06044378  0.
  -0.37112663]
 [ 0.          0.06164312  0.          0.          0.          0.
  -0.63589628  0.          0.          0.49811013 -0.35789701 -0.57144122
   0.        ]]


In [15]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = plt.subplot(111)
colors = ['blue', 'green', 'red', 'cyan',
         'magenta', 'yellow', 'black',
         'pink', 'lightgreen', 'lightblue',
         'gray', 'indigo', 'orange']
weights, params = [], []

for c in np.arange(-4, 6, dtype=float):
    lr = LogisticRegression(penalty='l1', 
                            C=10**c, 
                            random_state=0)
    lr.fit(X_train_std, y_train)
    weights.append(lr.coef_[1])
    params.append(10**c)
weights = np.array(weights)

for column, color in zip(range(weights.shape[1]), colors):
    plt.plot(params, weights[:, column],
             label=df_wine.columns[column+1], 
             color=color)

plt.axhline(0, color='black', linestyle='--', linewidth=3)
plt.xlim([10**(-5), 10**5])
plt.ylabel('koefisien bobot')
plt.xlabel('C')
plt.xscale('log')
ax.legend(loc='upper center',
         bbox_to_anchor=(1.3, 1.03),
         ncol=1, fancybox=True)
plt.show()

### Sekuensial feature selection
Sekeunsial feature selection adalah algoritma yang berasal dari keluarga greddy yang dignakan untuk mereduksi dimensi awal *d-dimensi* menjadi *k-dimensi*, dimana *k-dimensi* merupakan subspace shingga nilai *k<d*. Algoritma sekeunsial feature selection yang cukup klasik adalah ***Sequntial Backward Selection (SBS)*** tujuan algoritma ini adalah untuk mereduksi dimensi awal ke fitur subspace dengan peluruhan performa yang minimum, shingga SBS mampu meningkatkan prediksi suatu model.

Ide dibalik algoritma SBS sangat simple, SBS secara sekuensial mengilangkan feature dari full feature hingga fature mencapai jumlah yang di inginkan. Untuk menentukan feature mana yang akan dihilangkan perlu ditentukan fungsi kriteria nilai $J$ yang ingin kita minimalkan. Berikut ini adalah 4 langkah algoritma SBS
1. Inisiali Algoritma SBS dengan nilai $k=d$, dimana $d$ adalah dimensi dari ruang feature $X_d$
2. Tentukan fitur $X^-$ yang memaksimalkan $X^- = argmaxJ(X_k - x)$ dimana x
3. Hilangkan feature $x^-$ dari feature set $X_{k-1} := X_k - x^-

In [11]:
from sklearn.base import clone
from itertools import combinations
from sklearn.metrics import accuracy_score

class SBS():
    def __init__(self, estimator, k_features, 
                 scoring=accuracy_score, test_size=0.25, random_state=1):
        self.scoring = scoring
        self.estimator = clone(estimator)
        self.k_features = k_features
        self.test_size = test_size
        self.random_state = random_state
    
    
    def fit(X,y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=self.test_size,
                                                          random_state=self.random_state)
        dim = X_train.shape[1]
        self.indices = tuple(range(dim))
        self.subsets = [self.indices_]
        score = self._calc_score(X_train, y_train,
                                X_test, y_test, self.indices_)
        self.scores_ = [score]
        
        while dim > self.k_features:
            scores = []
            subsets = []
            
            for p in combinations(self.indices_, r = dim-1):
                score = self._calc_score(X_train, y_train,
                                        X_test, y_test, p)
                scores.append(score)
                subsets.append(p)
            
            best = np.argmax(scores)
            self.indices_ = subsets[best]
            self.subsets_.append(self.indices_)
            dim -= 1
            
            self.scores_.append(scores[best])
        self.k_score = self.scores_[-1]
        
        return self
    
    
    def transform(self, X):
        return X[:, self.indices_]
    
    
    def _calc_score(self, X_train, y_train,
                   X_test, y_test, indices):
        self.estimator.fit(X_train[:, indices], y_train)
        y_pred = self.estimator.predict(X_test[:, indices])
        score = self.scoreing(y_test, y_pred)
        return score

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=2)
sbs = SBS(knn, k_feature=1)
sbs.fit(X_train_std, y_train)